# ?0_Split_Videos.ipynb

** 'source activate python27 before use' 

Original code from Carolyn Parkinson & modified for personal use

Creates fslroi command to split whole run into individual videos

Input: 

* Run Ref File
    
    *  ./data/behavioraldata/scanfiles/239_run_ref.csv
    * Contains "sub_id, trial, video_number, self_other, neg_pos, scan_start, video_start, video_start_rts, run"
   
* Run File
    
Output: 
   
* Video run file

In [1]:
import os
import subprocess
import pandas as pd

from nilearn import input_data
from nilearn import image
from nilearn import plotting

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# define a function to execute shell commands
def sh(c):
    '''
    run shell commands
    '''
    subprocess.call(c, shell = True)
    
import math

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

In [ ]:
# get subject data

subjects = ['145', '157', '168', '184', '186', '200', '214', '216', '219', '220', '222', '223', '229', '245', '249', '253', '257', '263', '265', '267', '270', '277', '280', '282', '288']
for sub in subjects: 
    subject = "sub-" + sub
    for run in range(1,5): 

        ############# Set Filepaths #############
        data_filepath = os.path.join("./data/", subject, "func")

        reference_filepath = ("../EW_EmpInt_behav/data/behavioraldata/")
        run_ref_filepath = os.path.join(reference_filepath, "scanfiles", sub + "_run_ref.csv")
        run_duration_filepath = os.path.join(reference_filepath, "scanfiles_derivatives/rundata/video_timing", "video_duration_reference.csv")

        try: 
            
            ############# Load Reference Information and Calculate TRS #############
            # load reference file information
            run_ref = pd.DataFrame.from_csv(run_ref_filepath)
            run_duration = pd.DataFrame.from_csv(run_duration_filepath).reset_index()
            run_ref = pd.merge(run_ref, run_duration, on='video_number')

            # Get data only for relevant epi run
            run_ref = run_ref[run_ref.run == run]
            run_ref['video_number'] = run_ref['video_number'].astype(int)

            # Calculate trs
            # add three to account for hemodynamic lag
            run_ref['video_start_tr'] = \
                [round_up_to_even(x)/2 + 3 + 5 for x in run_ref['video_start_rts']] ## added + 5 to get rid of first three TRs

            run_ref['video_duration_tr'] = run_ref['duration']/2 - 3 #Get rid of last 3 TRs as well


            ############# Split Runs into 4 Videos #############
            # Split each epi into four videos
            for index, video in enumerate(run_ref['video_number'].tolist()): 

                run_filename = "sub-" + sub + "_task-emp_run-0" + str(run) + "_bold_space-MNI152NLin2009cAsym_preproc.nii.gz"
                bold_input_filename = os.path.join(data_filepath, run_filename)

                video_filename = ("video_{video}.nii.gz").format(video=str(video))
                bold_output_filename = os.path.join(data_filepath, video_filename)

                start = run_ref.iloc[index]['video_start_tr']
                duration = run_ref.iloc[index]['video_duration_tr']

                sh_cmd = ("""fslroi \
                      {bold_input_filename}\
                      {bold_output_filename}\
                      {start}\
                      {duration}""").format(bold_input_filename=bold_input_filename, bold_output_filename=bold_output_filename, start=start, duration=duration)
                sh(sh_cmd)
                print(sh_cmd) 


            ############# Plot Video Splitting #############
            
            #Load original and cut data
            bold_input = image.smooth_img(bold_input_filename, fwhm=3)
            bold_output= image.smooth_img(bold_output_filename, fwhm=3)

            print(bold_input.shape)
            print(bold_output.shape)

            #auditiory cortex seed
            aud_coords = [(-56,-4,-10),(56,-4,-10)]
            labels = ['Auditory Cortex - L', 'Auditory Cortex - R']

            #Make mask of auditory cortex
            masker = input_data.NiftiSpheresMasker(
                aud_coords, radius=8,
                detrend=True, standardize=True,
                low_pass=0.1, high_pass=0.01, t_r=2,
                memory='nilearn_cache', memory_level=1, verbose=1)
            
            #Rename Labels
            time_series = masker.fit_transform(bold_input)
            for time_serie, label in zip(time_series.T, labels):
                plt.plot(time_serie, label=label)
            
            #Label Plots
            plt.axvspan(run_ref.iloc[0]['video_start_tr'],run_ref.iloc[0]['video_start_tr'] + run_ref.iloc[0]['video_duration_tr'], alpha =.2)
            plt.axvspan(run_ref.iloc[1]['video_start_tr'],run_ref.iloc[1]['video_start_tr'] + run_ref.iloc[1]['video_duration_tr'], alpha =.2)
            plt.axvspan(run_ref.iloc[2]['video_start_tr'],run_ref.iloc[2]['video_start_tr'] + run_ref.iloc[2]['video_duration_tr'], alpha =.2)
            plt.axvspan(run_ref.iloc[3]['video_start_tr'],run_ref.iloc[3]['video_start_tr'] + run_ref.iloc[3]['video_duration_tr'], alpha =.2)
            plt.title('Aud Time Series'); plt.xlabel('TR number'); plt.ylabel('Normalized signal'); plt.legend(); plt.tight_layout()

            #Safe as png file
            plt.savefig(data_filepath + "/run-" +  str(run) + ".png")
            plt.gcf().clear()

        except:
            pass

In [15]:
# Concatenate videos into two runs (self and other)
# Self = videos 1, 2, 3, 4, 9, 10, 11, 12
# Other =videos 4, 6, 7, 8, 13, 14, 15, 16

subjects = ['145', '157', '168', '184', '186', '200', '214', '216', '219', '220', '222', '223', '229', '245', '249', '253', '257', '263', '265', '267', '270', '277', '280', '282', '288']
for sub in subjects: 
    subject = "sub-" + sub

    # set filepaths
    data_filepath = os.path.join("./data/", subject, "func/")

    # output filename
    self_concat_filename = "{}_self_concat.nii.gz".format(subject)

    # specify and execute an AFNI command to concatenate all runs' preprocessed data
#    tcat_cmd = ("3dTcat -session . -prefix {} "
#        "video_1.nii.gz video_2.nii.gz video_3.nii.gz video_4.nii.gz video_9.nii.gz video_10.nii.gz video_11.nii.gz video_12.nii.gz ").format(self_concat_filename)
#    print('pushd ' + data_filepath)
#    print(tcat_cmd)
#    print('popd')


    other_concat_filename = "{}_other_concat.nii.gz".format(subject)

        # specify and execute an AFNI command to concatenate all runs' preprocessed data
    tcat_cmd = ("3dTcat -session . -prefix {} "
        "video_5.nii.gz video_6.nii.gz video_7.nii.gz video_8.nii.gz video_13.nii.gz video_14.nii.gz video_15.nii.gz video_16.nii.gz ").format(other_concat_filename)
    print('pushd ' + data_filepath)
    print(tcat_cmd)
    print('popd')

pushd ./data/sub-145/func/
3dTcat -session . -prefix sub-145_other_concat.nii.gz video_5.nii.gz video_6.nii.gz video_7.nii.gz video_8.nii.gz video_13.nii.gz video_14.nii.gz video_15.nii.gz video_16.nii.gz 
popd
pushd ./data/sub-157/func/
3dTcat -session . -prefix sub-157_other_concat.nii.gz video_5.nii.gz video_6.nii.gz video_7.nii.gz video_8.nii.gz video_13.nii.gz video_14.nii.gz video_15.nii.gz video_16.nii.gz 
popd
pushd ./data/sub-168/func/
3dTcat -session . -prefix sub-168_other_concat.nii.gz video_5.nii.gz video_6.nii.gz video_7.nii.gz video_8.nii.gz video_13.nii.gz video_14.nii.gz video_15.nii.gz video_16.nii.gz 
popd
pushd ./data/sub-184/func/
3dTcat -session . -prefix sub-184_other_concat.nii.gz video_5.nii.gz video_6.nii.gz video_7.nii.gz video_8.nii.gz video_13.nii.gz video_14.nii.gz video_15.nii.gz video_16.nii.gz 
popd
pushd ./data/sub-186/func/
3dTcat -session . -prefix sub-186_other_concat.nii.gz video_5.nii.gz video_6.nii.gz video_7.nii.gz video_8.nii.gz video_13.nii.gz